In [1]:
# import and start sparknlp session
import sparknlp
sparknlp.start(spark32=True)

22/04/14 05:17:27 WARN Utils: Your hostname, DSGPU05 resolves to a loopback address: 127.0.1.1; using 10.10.11.64 instead (on interface eno1)
22/04/14 05:17:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark-3.2.1-bin-hadoop3.2/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/lbridges/.ivy2/cache
The jars for the packages stored in: /home/lbridges/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp-spark32_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a978faff-5fd4-42f4-9b33-30cebd614d77;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp-spark32_2.12;3.4.2 in central
	found com.typesafe#config;1.4.1 in central
	found org.rocksdb#rocksdbjni;6.5.3 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.603 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.code.findbugs#annotations;3.0.1 in central
	found net.jcip#jcip-annotations;1.0 in central
	found com.google.code.findbugs#jsr305;3.0.1 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found or

In [2]:
# start spark session configured for spark nlp
from pyspark.sql import SparkSession

spark = SparkSession.builder \
     .master('local[*]') \
     .appName('Spark NLP') \
     .config('spark.jars.packages', 
             'com.johnsnowlabs.nlp:spark-nlp_2.12:2.3.5') \
     .getOrCreate()


In [3]:
# import stopwords dictionary
from nltk.corpus import stopwords
eng_stopwords = stopwords.words('english')

In [4]:
# import transformers
from sparknlp.base import Finisher, DocumentAssembler
from sparknlp.annotator import (Tokenizer, Normalizer,
                                LemmatizerModel, StopWordsCleaner)
from pyspark.ml import Pipeline

In [5]:
# define pipeline
documentAssembler = DocumentAssembler() \
     .setInputCol('value') \
     .setOutputCol('document')
tokenizer = Tokenizer() \
     .setInputCols(['document']) \
     .setOutputCol('token')
# note normalizer defaults to changing all words to lowercase.
# Use .setLowercase(False) to maintain input case.
normalizer = Normalizer() \
     .setInputCols(['token']) \
     .setOutputCol('normalized') \
     .setLowercase(True)
# note that lemmatizer needs a dictionary. So I used the pre-trained
# model (note that it defaults to english)
lemmatizer = LemmatizerModel.pretrained() \
     .setInputCols(['normalized']) \
     .setOutputCol('lemma')
stopwords_cleaner = StopWordsCleaner() \
     .setInputCols(['lemma']) \
     .setOutputCol('clean_lemma') \
     .setCaseSensitive(False) \
     .setStopWords(eng_stopwords)
# finisher converts tokens to human-readable output
finisher = Finisher() \
     .setInputCols(['clean_lemma']) \
     .setCleanAnnotations(False)

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[ | ]lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
Download done! Loading the resource.
[ / ]

[OK!]


In [6]:
# assemble pipeline
pipeline = Pipeline() \
     .setStages([
           documentAssembler,
           tokenizer,
           normalizer,
           lemmatizer,
           stopwords_cleaner,
           finisher
     ])

In [7]:
# import data

from pyspark.sql.functions import monotonically_increasing_id, row_number
from pyspark.sql import Window, SQLContext
sqlContext = SQLContext(spark)


path = 'Combined'
data = spark.read.text(path, wholetext=True)

# create column identifying whether word is from fanfic or literature text file
books = ["fanfic", "literature"]
b = sqlContext.createDataFrame([(l,) for l in books], ['Book'])
data = data.withColumn("row_idx", row_number().over(Window.orderBy(monotonically_increasing_id())))
b = b.withColumn("row_idx", row_number().over(Window.orderBy(monotonically_increasing_id())))

data = b.join(data, b.row_idx == data.row_idx).\
    drop("row_idx")


/opt/spark/python/lib/pyspark.zip/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.


In [8]:
# transform text with the pipeline

processed_data = pipeline.fit(data).transform(data)

In [9]:
# add column to input explode finished clean lemma list
from pyspark.sql.functions import explode, col
scifi_words = processed_data.withColumn('exploded_text', 
                               explode(col('finished_clean_lemma')))

In [10]:
# transform list of words found in fanfic text into dictionary with counts
counts_ff = scifi_words.filter(scifi_words['Book'] == 'fanfic')
counts_ff2 = counts_ff.groupby('exploded_text').count()
counts_ff_pd = counts_ff2.toPandas()
ff_dict = {counts_ff_pd.loc[i, 'exploded_text']: 
                counts_ff_pd.loc[i, 'count'] 
                for i in range(counts_ff_pd.shape[0])}

22/04/14 05:19:14 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/04/14 05:19:14 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/04/14 05:19:15 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/04/14 05:19:15 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/04/14 05:19:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/04/14 05:19:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [11]:
import operator
import pandas as pd
# dict(sorted(ff_dict.items(), key=lambda item: item[1]))
ff_sort = dict(sorted(ff_dict.items(), key=operator.itemgetter(1), reverse = True))
pd_ff_sort = pd.DataFrame.from_dict(ff_sort, orient='index')
pd_ff_sort.to_csv("ffsorted.csv")

In [12]:
# transform list of words found in lit text into dictionary with counts

counts_lit = scifi_words.filter(scifi_words['Book'] == 'literature')
counts_lit2 = counts_lit.groupby('exploded_text').count()
counts_lit_pd = counts_lit2.toPandas()
lit_dict = {counts_lit_pd.loc[i, 'exploded_text']: 
                counts_lit_pd.loc[i, 'count'] 
                for i in range(counts_lit_pd.shape[0])}

22/04/14 05:19:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/04/14 05:19:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/04/14 05:19:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/04/14 05:19:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/04/14 05:19:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/04/14 05:19:45 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [14]:
# Helper Functions

def term_frequency(BoW_dict):
     tot_words = sum(BoW_dict.values()) #calcs total number of words in dict
     freq_dict = {word: BoW_dict[word]/tot_words #creates dict of word:freq in each dict
                  for word in BoW_dict.keys()}
     return freq_dict
from math import log
def inverse_document_frequency(list_of_dicts):
    tot_docs = len(list_of_dicts) #calcs total number of docs in list
    words = set([w for w_dict in list_of_dicts #creates set of words in all docs in list
                   for w in w_dict.keys()])
    idf_dict = {word: log(1.0 + float(tot_docs)/ #calcs log(tot_docs/doc freq) for each word in words
                      (1.0 + sum([1 for w_dict in list_of_dicts 
                              if word in w_dict.keys()]))) 
                    for word in words}
    return idf_dict
def tf_idf(list_of_dicts):
     words = set([w for w_dict in list_of_dicts #creates set of words in all docs in list
                  for w in w_dict.keys()])
     tf_idf_dicts = []
     idfs = inverse_document_frequency(list_of_dicts) #creates dict of words:idf_score
     for i, w_dict in enumerate(list_of_dicts): # for each dict in list_of_dicts
          w_dict.update({word: 0 for word in words # add word:0 if not in set words
                         if word not in w_dict.keys()})
          tf = term_frequency(w_dict) #calc tf for dict
          tf_idf_dicts.append({word: tf[word]*idfs[word] # append tf-idf to tf_idf_dicts 
                               for word in words})
     return tf_idf_dicts

In [15]:
# perform tf_idf on dictionaries to return most prominent,unique words in each dictionary

list_of_word_dicts = [ff_dict, lit_dict]
tf_idf_by_book_list = tf_idf(list_of_word_dicts)
tf_idf_by_book_dict = {c: tf_dict 
           for c, tf_dict in zip(books, tf_idf_by_book_list)}

In [18]:
# export lit dictionary as csv (the tf_idf algorithm switches the dictionary from fanfic to lit, and vice versa for some reason so they are renamed appropriately)

import operator
import pandas as pd
sorted_lit = dict( sorted(tf_idf_by_book_dict['fanfic'].items(), key=operator.itemgetter(1),reverse=True))
pd_lit = pd.DataFrame.from_dict(sorted_lit, orient='index')
pd_lit.to_csv("pdlit.csv")


In [25]:
sorted_ff = dict( sorted(tf_idf_by_book_dict['literature'].items(), key=operator.itemgetter(1),reverse=True))
pd_ff = pd.DataFrame.from_dict(sorted_ff, orient='index')
pd_ff.to_csv("pdff.csv")

{'say': 0.005438588698978546,
 'john': 0.004820567255912802,
 'iris': 0.004048040452080623,
 'go': 0.003800831874854325,
 'jungkook': 0.0037317826550018226,
 'martha': 0.003647922370619759,
 'jimin': 0.0034382716596646004,
 'get': 0.0033682168647083044,
 'know': 0.0032137115039418684,
 'like': 0.003182810431788581,
 'one': 0.003151909359635294,
 'think': 0.0028428986381024215,
 'back': 0.002719294349489273,
 'ship': 0.0025647889887228372,
 'look': 0.0025647889887228372,
 'make': 0.0025338879165695497,
 'see': 0.0024720857722629755,
 'well': 0.0024720857722629755,
 'take': 0.002410283627956401,
 'would': 0.0022866793393432525,
 'come': 0.002255778267189965,
 'ask': 0.002224877195036678,
 'alec': 0.002180367393933649,
 'time': 0.002101272906423529,
 'still': 0.002070371834270242,
 'way': 0.002070371834270242,
 'im': 0.002039470762116955,
 'man': 0.0020085696899636678,
 'right': 0.0020085696899636678,
 'even': 0.0019776686178103802,
 'eye': 0.001915866473503806,
 'dont': 0.001823163257043

In [16]:
tf_idf_by_book_dict

{'fanfic': {'cash': 4.617424060074036e-06,
  'keen': 5.0123632328297587e-05,
  'strand': 4.617424060074036e-06,
  'intermediary': 1.2530908082074397e-05,
  'greatgreatgreatgrandfather': 6.265454041037198e-06,
  'insensibly': 6.265454041037198e-06,
  'canoe': 0.00010024726465659517,
  'defy': 1.2530908082074397e-05,
  'echo': 2.7704544360444218e-05,
  'barge': 6.265454041037198e-06,
  'alert': 2.5061816164148793e-05,
  'immunity': 1.2530908082074397e-05,
  'allwhile': 6.265454041037198e-06,
  'horrible': 0.00019393181052310954,
  'york': 9.234848120148073e-06,
  'multiplication': 6.265454041037198e-06,
  'wordsstrictly': 6.265454041037198e-06,
  'certifiable': 6.265454041037198e-06,
  'number': 0.0003509242285656268,
  'ardour': 6.891999445140918e-05,
  'environ': 6.265454041037198e-06,
  'condescend': 1.2530908082074397e-05,
  'directness': 6.265454041037198e-06,
  'set': 0.00034630680450555267,
  'glassesthought': 6.265454041037198e-06,
  'sacrilegiously': 6.265454041037198e-06,
  'im